# Demo Usage
For the demo usage, I apply very simple (which doesn't work very well) neural network and send their information to notion page. Sample Notion page is [here](https://www.notion.so/DeepNotion-Sample-Page-3709487e93694d0f97d432335ae16100)

# Login

My configuration file is hidden (for sure, since `token_v2` is my privacy) but I'll show what's insde my .yml's `url`: https://www.notion.so/DeepNotion-Sample-Page-3709487e93694d0f97d432335ae16100

In [1]:
from build import *
from time_utils import *

root, client = login() # LOGIN. dictionary of config or path of your .yml
page = make_page(root, title='Database '+today())

[login] 0.7 sec
[make_page] 5.3 sec


# Make Database on to that page

In [2]:
db = make_db(page, title='Breast Cancer')

[login] 1.0 sec
[make_db] 10.9 sec


# Very Simple PyTorch Example

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [4]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
trn_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float), torch.tensor(y_train))
trn_dl = DataLoader(trn_ds, batch_size=8)

tst_ds = TensorDataset(torch.tensor(X_test, dtype=torch.float), torch.tensor(y_test))
tst_dl = DataLoader(tst_ds, batch_size=8)

In [6]:
class ANN(nn.Module):
    
    def __init__(self):
        super(ANN, self).__init__()
        
        self.fc1 = nn.Linear(30, 10)
        self.fc2 = nn.Linear(10, 1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        
        return x

In [7]:
model = ANN()

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
trn_loss, tst_loss = [], []
for e in range(10):
    
    preds = []
    for x, y in trn_dl:
        
        y_pred = model(x)
        loss = loss_fn(y_pred, y.float())
        preds.append((y_pred > 0).type(torch.int).squeeze(1) - y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    preds = torch.cat(preds)
    trn_acc = 1 - torch.sum(preds).type(torch.float) / len(preds)
    trn_loss = loss.item()
        
    preds = []
    for x, y in tst_dl:
        
        y_pred = model(x)
        loss = loss_fn(y_pred, y.float())
        preds.append((y_pred > 0).type(torch.int).squeeze(1) - y)
        
    preds = torch.cat(preds)
    tst_acc = 1 - torch.sum(preds).type(torch.float) / len(preds)
    tst_loss = loss.item()
    
    # This part is sending the result to the notion
    # ... and the model works horribly hehe
    # I recommend you to send these informations not every epochs
    # but rather 5 or 10 epochs at a time ...
    # Since...
    # 1. It takes some time when writing
    # 2. If it happens too much, notion server will block you :(
    data = {
        'Epochs': f'{e}',
        
        'Train Loss': trn_loss,
        'Train Acc': float(trn_acc),
        
        'Test Loss': tst_loss,
        'Test Acc': float(tst_acc),
    }
    try:
        write_db(db, data)
        
    except HTTPError:
        pass

[write_db] 52.6 sec
[write_db] 22.4 sec
[write_db] 10.3 sec
[write_db] 13.1 sec
[write_db] 12.7 sec
[write_db] 28.1 sec
[write_db] 14.2 sec
[write_db] 11.1 sec
[write_db] 15.6 sec
[write_db] 14.9 sec
